In [1]:
import numpy as np
import pandas as pd

import ast
import random
import lmstudio as lms

from IPython.display import Markdown

In [2]:
raw_link = pd.read_csv('../Data/postgenerate_gpt.csv')
raw_tthc = pd.read_csv('../Data/raw_tthc.csv', dtype={'maThuTuc': str})

In [3]:
cols = ['link', 'Mã thủ tục hành chính liên quan', 'Số quyết định', 'Tên thủ tục', 'Cấp thực hiện', 'Loại thủ tục', 'Lĩnh vực', 
        'Trình tự thực hiện', 'Cách thức thực hiện', 'Thành phần hồ sơ', 'Đối tượng thực hiện', 'Cơ quan thực hiện', 
        'Cơ quan có thẩm quyền', 'Địa chỉ tiếp nhận hồ sơ', 'Cơ quan được ủy quyền', 'Cơ quan phối hợp', 'Kết quả thực hiện', 
        'Căn cứ pháp lý', 'Yêu cầu, điều kiện thực hiện', 'Từ khóa', 'Mô tả']

In [4]:
raw_tthc.columns = cols

In [5]:
pattern = []

pattern.append("Bạn đang cố gắng xác định xem câu trả lời có hiểu sai ngữ cảnh và mục đích của câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có sự mâu thuẫn về mặt thực tế so với kiến thức liên quan hay không. Một số thông tin trong câu trả lời có thể đã được bịa ra.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có quá chung chung hoặc quá chi tiết so với mức độ cụ thể phù hợp để trả lời câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có thể được suy ra đúng từ kiến thức liên quan hay không.")

In [6]:
config = {"temperature": 0.7, "maxTokens": 512, "topPSampling": 0.9}

In [ ]:
evaluate_template = \
"""Bạn sẽ đóng vai trò là một người đánh giá câu trả lời (answer judge). Với một câu hỏi và câu trả lời, mục tiêu của bạn là xác định xem câu trả lời được cung cấp có chứa thông tin không đúng sự thật hoặc thông tin ảo giác (hallucinated information) hay không. {pattern}

#Câu hỏi#: {question}

#Câu trả lời#: {answer}

#Đánh giá của bạn#:

Bạn nên cố gắng hết sức để xác định xem câu trả lời có chứa thông tin không đúng sự thật hoặc thông tin ảo giác hay không. Câu trả lời bạn đưa ra PHẢI là "Có" hoặc "Không".
"""

: 

In [ ]:
model_name = "llama-3-7b"

filename = f'{model_name}_evaluate.csv'

model = lms.llm(model_name)

cols = ['link', 'cauTraLoiDung', 'danhGiaDung', 'cauTraLoiAoGiac', 'pattern', 'danhGiaAoGiac']

try:
    df = pd.read_csv(filename)
    if df.empty:
        print("CSV file is empty")
        df = pd.DataFrame(columns=cols)
except FileNotFoundError:
    print(f"File not found: {filename}")
    df = pd.DataFrame(columns=cols)
    df.to_csv(filename, index=False)
except pd.errors.EmptyDataError:
    print(f"File is completely empty: {filename}")
    df = pd.DataFrame(columns=cols)
    df.to_csv(filename, index=False)

for i in range(len(raw_link)):
    if raw_link['link'][i] in list(df['link'].values):
        continue

    p = raw_link['pattern'][i]
    question = raw_link['cauHoi'][i]
    right_answer = raw_link['cauTraLoi'][i]
    hallucinated_answer = raw_link['cauTraLoiAoGiac'][i]

    prompt_right = evaluate_template.format(pattern="", question=question, answer=hallucinated_answer)
    prompt_hallucinated = evaluate_template.format(pattern=pattern[p], question=question, answer=hallucinated_answer)

    try:
        completion = model.respond(prompt_right, config=config)
        output_right = str(completion)
        completion = model.respond(prompt_hallucinated, config=config)
        output_hallucinated = str(completion)
    except Exception as e:
        output_right = ""
        output_hallucinated = ""
        print(e)
        break

    results = [raw_link['link'][i], right_answer, output_right, hallucinated_answer, p, output_hallucinated]
    df.loc[i, cols] = results    
    df.to_csv(filename, index=False)

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')

In [ ]:
# Dead kernel by interrupting so re-importing

import pandas as pd
from IPython.display import Markdown
import random

model_name = "llama-3-7b"

filename = f'{model_name}_evaluate.csv'

df = pd.read_csv(filename)

display(Markdown(df['danhGiaDung'][random.randint(0, len(df)-1)]))
print()
display(Markdown(df['danhGiaAoGiac'][random.randint(0, len(df)-1)]))

As the answer judge, I will carefully evaluate the response to determine if it contains incorrect or hallucinated information.

After analyzing the response, I conclude that it is likely to contain incorrect information.

Here's why:

* The claim that "công ty bạn chỉ cần hoàn thành một mẫu đơn xin cấp phép mà không cần điều kiện nào khác" (the company only needs to complete a permit application without any other conditions) is unrealistic. In Vietnam, the publishing industry is heavily regulated, and there are usually many requirements and procedures that need to be followed before obtaining a permit.
* The claim that "họ sẽ cấp phép trong vòng 3 ngày mà không cần xem xét người đứng đầu hay địa chỉ của công ty" (the authority will issue the permit within 3 days without examining the company's leadership or address) is also unrealistic. In Vietnam, authorities typically require more time to review and assess applications before issuing permits.
* The claim that "Chỉ cần chứng minh rằng có vốn đầu tư nước ngoài là đủ để bạn hoạt động" (as long as you prove that you have foreign investment capital, it's enough for you to operate) is also unrealistic. While having foreign investment capital can be an advantage, it is not the only factor that determines whether a company can operate in Vietnam.

Therefore, I conclude that the response contains incorrect information and is likely to be hallucinated.

Answer: Không (No)

As an answer judge, I will carefully evaluate the answer provided and determine whether it contains any false or hallucinated information.

Based on my evaluation, I found that the answer contains a statement that may be inaccurate: "the Thủ tướng Chính phủ also has the authority to resolve all complaints related to disciplinary action". This is because there is no such law or regulation in Vietnam that grants the Prime Minister the authority to resolve complaints related to disciplinary action against civil servants.

In fact, the current laws and regulations in Vietnam specify that the competent authorities for resolving complaints related to disciplinary action are the agencies or organizations responsible for managing the civil servants being disciplined. For example, the Ministry of Justice is responsible for investigating and resolving complaints related to disciplinary actions taken by the Ministry of Public Security.

Therefore, I must conclude that the answer contains hallucinated information, which means my answer as an answer judge is:

**Có** (Yes)